## Level 5 Data Engineer
## Module 3 Topic 9


# Pip Installs

In [ ]:
!pip install pandasql
!pip install sqlalchemy==1.4.46
!pip install sweetviz
!pip install lazypredict

# Library Imports

In [ ]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import sweetviz as sv
import os
import seaborn as sns
import warnings
import logging

warnings.filterwarnings('ignore')

In [ ]:
# font size globally defined
font = {'family' : 'normal',
'weight' : 'bold',
'size' : 14}

matplotlib.rc('font', **font)

In [ ]:
import kagglehub

# Download latest version
data_path = kagglehub.dataset_download("uciml/default-of-credit-card-clients-dataset")

print("Path to dataset files:", data_path)

In [ ]:
os.listdir(data_path)

In [ ]:
# load our data
data = pd.read_csv(os.path.join(data_path, 'UCI_Credit_Card.csv'))
data = data[['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE',
             'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
             'default.payment.next.month']]
data.rename(columns={
    'ID':'CLIENT_ID',
    'default.payment.next.month':'DEFAULT'}, inplace=True)
data.head()

Data dictionary

- **CLIENT_ID**: Unique identifier for each client.
- **LIMIT_BAL**: The total credit amount (in pound sterling) granted to the client, including both individual and supplementary credit.
- **SEX**: (1 = male, 2 = female).
- **EDUCATION**: Client's highest level of education (1 = university, 2 = college, 3 = apprenticeship, 4 = unknown, 5 = GCSE, 6 = unknown).
- **MARRIAGE**: Client's marital status (1 = married, 2 = single, 3 = other).
- **AGE**: Client's age in years.
- **PAY_0**: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above)
- **PAY_2**: Repayment status in August 2005, with the same scale as PAY_0.
- **PAY_3**: Repayment status in July 2005, with the same scale as PAY_0.
- **PAY_4**: Repayment status in June 2005, with the same scale as PAY_0.
- **PAY_5**: Repayment status in May 2005, with the same scale as PAY_0.
- **PAY_6**: Repayment status in April 2005, with the same scale as PAY_0.
- **Default**: Credit card default states (0=No, 1=Yes).

In [ ]:
# value counts of default
plt.style.use('fivethirtyeight')
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

fig, ax = plt.subplots(figsize=(6,3))

data['DEFAULT'].value_counts(normalize=True).plot(kind='barh', ax=ax)
print(data['DEFAULT'].value_counts(normalize=True) *100)

ax.set(xlabel='Proportion 0-1', ylabel='Default Label', title='Credit Card Default Distribution');

# Data Profiling

In [ ]:
Report = sv.analyze(data)

Report.show_notebook()

# Train-Test Split

In [ ]:
# cleaning up the data
lower_names = []

for column_name in data.columns:
    lower_names.append(column_name.lower())

data.columns = lower_names

In [ ]:
data.columns

In [ ]:
# what is our distribution for the target variable in the original data?

data['default'].value_counts(normalize=True) * 100

In [ ]:
# split the data into train - test set
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, test_size=0.3,
                                     random_state=1234,
                                     stratify=data['default'])

print(df_train['default'].value_counts(normalize=True))
print(df_test['default'].value_counts(normalize=True))

In [ ]:
# what did we do?
print(data.shape)
print(df_train.shape)
print(df_test.shape)

# Exploratory Data Analysis (EDA)

In [ ]:
df_train['default'].mean()

# Feature Engineering / Extraction

It is the formal process of taking raw data and creating new columns and measures by combining existing columns or applying formulas on them.



In [ ]:
# helper function to add together positive delay values
def sum_positive(series):
    filter_condition = series > 0
    return series[filter_condition].sum()


In [ ]:
data.columns

In [ ]:
# calculate features for df train

df_train['avg_delay'] = (df_train[['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']]
                         .apply(lambda x: x[x > 0].mean(), axis=1))


df_train['avg_delay'].fillna(0, inplace=True)

df_train.head()

In [ ]:
# create extra information - df test
df_test['avg_delay'] = (df_test[['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']]
                         .apply(lambda x: x[x > 0].mean(), axis=1))

df_test['avg_delay'].fillna(0, inplace=True)

df_test.head()

In [ ]:
# total payment delay
df_train['total_delay'] = (df_train[['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']]
                         .apply(sum_positive, axis=1))

df_train.head()

| Feature    | Value  | Description                                              |
|------------|--------|----------------------------------------------------------|
| CLIENT_ID  | 7188   | Unique identifier for the client                         |
| LIMIT_BAL  | 20000  | Total credit amount (in NT dollars)                      |
| SEX        | 1      | Gender (1 = male)                                        |
| EDUCATION  | 2      | Highest level of education (2 = university)              |
| MARRIAGE   | 2      | Marital status (2 = single)                              |
| AGE        | 25     | Age in years                                             |
| PAY_0      | 1      | Repayment status in September 2005 (1 month delayed)     |
| PAY_2      | 2      | Repayment status in August 2005 (2 months delayed)       |
| PAY_3      | 2      | Repayment status in July 2005 (2 months delayed)         |
| PAY_4      | 2      | Repayment status in June 2005 (2 months delayed)         |
| PAY_5      | 2      | Repayment status in May 2005 (2 months delayed)          |
| PAY_6      | 0      | Repayment status in April 2005 (revolving credit or no transactions) |
| DEFAULT    | 0      | Credit card default status (1 = Yes)                     |
| avg_delay  | 1.80   | Average delay in payments across all months              |
| total_delay| 9      | Total delay in payments across all months                |


In [ ]:
# total delay with the test set
df_test['total_delay'] = (df_test[['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']]
                         .apply(sum_positive, axis=1))

df_test.head()

## Pandas Groupby Function

The Pandas `groupby` function is a powerful method used for grouping DataFrame rows based on the values of one or more columns. It enables you to perform various aggregation operations on the grouped data, such as computing the sum, mean, count, or other custom aggregation functions.

### Syntax

`DataFrame.groupby(by, axis, level, as_index, sort, group_keys, squeeze, observed)`

- `by`: The column(s) to group by. Can be a single column, a list of columns, or a dictionary mapping columns to group names.
- `axis`: The axis to group along, either 0 (default) for rows or 1 for columns.
- `sort`: If `True` (default), the group keys will be sorted. If `False`, the group keys will not be sorted, which may improve performance.
- `group_keys`: If `True` (default), the group keys will be included in the result. If `False`, the group keys will not be included.
- `observed`: If `True`, only show observed values for categorical groupers. If `False` (default), show all values.

### Basic Example

```python
import pandas as pd

# Sample DataFrame
data = {'Category': ['A', 'B', 'A', 'B', 'A', 'B'],
        'Value': [10, 20, 30, 40, 50, 60]}
df = pd.DataFrame(data)

# Group by Category
grouped = df.groupby('Category')

# Compute the sum of the Value column for each group
result = grouped['Value'].sum()


In [ ]:
# hypothesis: customers who delay more often and over longer durations default more!
agg_df1 = df_train.groupby('total_delay').agg(count=("total_delay", "size"),
                                    mean_target=("default", "mean")).reset_index()

agg_df1

In [ ]:
# visualise
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots(figsize=(12,6))

# create secondary y-axis
ax2 = ax.twinx()

ax.bar(list(agg_df1['total_delay']), list(agg_df1["count"]))
ax2.plot(list(agg_df1['mean_target']), color='r', linewidth=3)

ax.grid(False)

In [ ]:
# average delay
agg_df2 = df_train.groupby('avg_delay').agg(count=("avg_delay", "size"),
                                    mean_target=("default", "mean")).reset_index()

agg_df2.head()

# Feature Selection - What do we include in our model?

In [ ]:
# what are the irrelevant columns?
df_train.columns

In [ ]:
# let us look at a correlation matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=0.8)

correlation_matrix = df_train.drop('client_id', axis=1).corr()

fig, ax = plt.subplots(figsize=(10,6))

mask = np.zeros_like(correlation_matrix, dtype=bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(correlation_matrix, mask=mask, ax=ax, cmap='YlGnBu', annot=True)

# Model Building - Predictive Analytics

Step 1) Drop all the irrelevant columns and split X_train, X_test

Step 2) Select the right model and import it and use it to train

Step 3) We train on the training sets & test on the X_test (generating y_pred - basically running the model)

Step 4) We evaluate the performance of our y_pred against the true values of y_test (i.e., generate the accuracy and loss metrices)

Step 5) Model diagnostics - creating reports on the model performance

Step 6) Conclusion model is not good enough and we need to fine-tune / or model is good enough and we can publish!

In [ ]:
# what is the right model to pick?
df_train['default'].unique()

**Our Prediction Problem is a Classification Task! Binary Classification**

In [ ]:
# what is the effect of limit_balance
# are customer with high limit balance less likely to default?

# Distribution of LIMIT_BAL
import matplotlib.ticker as mticker
fig, ax = plt.subplots(figsize=(10, 6))
sns.histplot(data=df_train, x='limit_bal', kde=True, hue="default",
             bins=20, multiple="dodge", ax=ax, shrink=0.8)

# Format x-axis with dollar sign and "k" for thousands
formatter = mticker.FuncFormatter(lambda x, pos: f"${int(x/1000)}k")
ax.xaxis.set_major_formatter(formatter)

# Set the title
ax.set_title('Credit Limit Distribution')

# Add arrow annotation
arrow_props = dict(facecolor='steelblue')
ax.annotate('Non-defaulting customers have a higher credit limit on average',
             xy=(220000, 2000), xytext=(400000, 2800),
             fontsize=12, color='dodgerblue', arrowprops=arrow_props)

plt.show()

In [ ]:
print(df_train.columns)

In [ ]:
# drop the irrelevant columns
# marriage, education, client_id, age, age_group
cols_to_drop = ['client_id', 'sex', 'education', 'marriage', 'age']

df_train.drop(cols_to_drop, axis=1, inplace=True)
df_test.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
df_train.head(20)

In [ ]:
# split the data further into X_train, X_test, y_train, y_test

X_train = df_train.drop('default', axis=1)
X_test = df_test.drop('default', axis=1)

y_train = df_train['default']
y_test = df_test['default']

In [ ]:
# we can inspect the arrays
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# what is X_train?
X_train.head()

In [ ]:
# binary balance helper

def binary_balance(target_train, target_test, plot_dist=True):
    '''
    Plot distribution of binary target
    '''
    if plot_dist:
        fig, axs = plt.subplots(ncols=2, figsize=(17, 5), sharey=True)
        target_train.value_counts(normalize = True).plot(kind="bar", ax=axs[0])
        axs[0].set_title("Training Set Distribution")
        axs[0].set_xticklabels(target_train.unique(),rotation=70)
        target_test.value_counts(normalize=True).plot(kind="bar", ax=axs[1])
        axs[1].set_title("Test Set Distribution")
        axs[1].set_xticklabels(target_test.unique(), rotation=70)
        plt.show()

In [ ]:
# compare the distribution of the two arrays
binary_balance(y_train, y_test)

In [ ]:
# lazypredict
from lazypredict.Supervised import LazyClassifier

# load the lazyclassifier engine - specify settings
clf = LazyClassifier(verbose=0, ignore_warnings=True,
                     custom_metric=None, predictions=True)

# run the lazyclassifier engine
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

models

In [ ]:
# for better readability we can sort the results
models.sort_values(by=['Accuracy', 'F1 Score'], axis=0, ascending=False)

In [ ]:
# run a random forest classifier algorithm!
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *

# step 1: initialise / load the model
classifier = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=4)

# step 2: train the model on X_train, y_train
classifier.fit(X_train, y_train)

# step 3: generate y_pred - using .predict()
y_pred = classifier.predict(X_test)

# step 4: evaluate the accuracy and compare y_pred to the true values y_test
accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Balanced Accuracy:", balanced_accuracy)
print("F1-score: ", f1)

Why are the results less than ideal? Potential Solutions?

1) Get more data: we haven't actually used all the features available in the dataset

2) Hyperparameter tuning and experimenting with different algorithms

3) Speak to domain experts to improve our feature engineering approach.
